In [1]:

# coding: utf-8

# In[12]:



import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn import preprocessing
import warnings
import datetime

warnings.filterwarnings("ignore")

import time

from utils import raw_data_path, dump_pickle

path = '../data/'
train_file = 'round1_ijcai_18_train_20180301.txt'
test_file = 'round1_ijcai_18_test_a_20180301.txt'



# def load_data():
#     train = pd.read_table(path + train_file, encoding='utf8', delim_whitespace=True)
#     test = pd.read_table(path + train_file, encoding='utf8', delim_whitespace=True)
#     df = pd.concat([train, test], axis=0, ignore_index=True)

def date_convert(data):
    # Transform into datetime format
    data['time'] = pd.to_datetime(data.context_timestamp, unit='s')

    # transform into Beijing datetime format
    data['realtime'] = data['time'].apply(lambda x: x + datetime.timedelta(hours=8))
    data['day'] = data['realtime'].dt.day
    data['hour'] = data['realtime'].dt.hour
    
    return data

def base_process(data):
    lbl = preprocessing.LabelEncoder()
    print("========================item==========================")
    # Divided into different category levels and LabelEncoder()
    '''
    item_id, item_category_list, item_property_list, item_brand_id, item_city_id, 
    item_price_level, item_sales_level, item_collected_level, item_pv_level
    '''
    for i in range(1, 3):
        data['item_category_list' + str(i)] = lbl.fit_transform(data['item_category_list'].map(
            lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else 'missing'))
    del data['item_category_list'] 
        
#     for i in range(10):
#         data['item_property_list' + str(i)] = lbl.fit_transform(data['item_property_list'].map(
#             lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''))
#     del data['item_property_list']
    # train_item_property = data['item_property_list'].str.split(';', expand=True).add_prefix('item_property_')
    # train_item_property.fillna('missing', inplace=True)
    # train_item_property = lbl.fit_transform(train_item_property)
    # data = pd.concat([data, train_item_property], axis=1)

    for col in ['item_id', 'item_brand_id', 'item_city_id']:
        data[col] = lbl.fit_transform(data[col])
    
    # Fill none with mean
    data['item_sales_level'][data.item_sales_level==-1] = None
    data['item_sales_level'].fillna(data['item_sales_level'].mean(), inplace=True)
    
    
    print("========================user==========================")
    # user_gender_id and user_occupation_id should be handled with one-hot
    data[data.user_age_level==-1]['user_age_level'] = None
    data['user_age_level'].fillna(data['user_age_level'].mode())
    data['user_age_level'] = data['user_age_level'].apply(lambda x: x%1000)
    
    data[data.user_star_level==-1]['user_star_level'] = None
    data['user_star_level'].fillna(data['user_star_level'].mean())
    data['user_star_level'] = data['user_star_level'].apply(lambda x: x%3000)
    
    
    print("=====================context==========================")
    data = date_convert(data)
    
    for i in range(5):
        data['predict_category_property' + str(i)] = lbl.fit_transform(data['predict_category_property'].map(
            lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''))
    del data['predict_category_property'] 
        
    print("=====================shop===============================")
    data['shop_score_service'][data.shop_score_service==-1] = None
    data['shop_score_service'].fillna(data['shop_score_service'].mean(), inplace=True)
    
    data['user_age_level'][data.user_age_level==-1] = None
    data['shop_score_delivery'].fillna(data['shop_score_delivery'].mean(), inplace=True)
    
    data['shop_score_description'][data.shop_score_description==-1] = None
    data['shop_score_description'].fillna(data['shop_score_description'].mean(), inplace=True)
    
    return data
    

if __name__ == "__main__":
    start = time.time()
    print("Load Data")
    train = pd.read_table(path + train_file, encoding='utf8', delim_whitespace=True)
    test = pd.read_table(path + test_file, encoding='utf8', delim_whitespace=True)
    train.drop_duplicates('instance_id', inplace=True)
    test.drop_duplicates('instance_id', inplace=True)
    print('the shape of train {}'.format(train.shape))
    print('the shape of test {}'.format(test.shape))
    len_train = train.shape[0]
    df = pd.concat([train, test], axis=0, ignore_index=True)
    print("Start doing preprocessing")
    
    df = base_process(df)
    dump_pickle(df, path=raw_data_path + 'df.pkl')
    
    train = df[(df['day'] >= 18) & (df['day'] <= 24)]
    print('the shape of train {}'.format(train.shape))
    dump_pickle(train, path=raw_data_path + 'train.pkl')
    
    test = df[df['day'] ==25]
    print('the shape of test {}'.format(test.shape))
    dump_pickle(test, path=raw_data_path + 'test.pkl')
    
    end = time.time()
    print("Preprocessing done and time elapsed %s" % (end-start))


Load Data
the shape of train (478087, 27)
the shape of test (18371, 26)
Start doing preprocessing
========================item==========================
========================user==========================
=====================context==========================
=====================shop===============================
the shape of train (478087, 36)
the shape of test (18371, 36)
Preprocessing done and time elapsed 28.34149408340454


In [6]:
 train

,,",instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,user_age_level,user_occupation_id,user_star_level,context_id,context_timestamp,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade,time,realtime,day,hour"
"0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;2636395404473730413;8070395809181082496;6434796455031995313;7839592306500064003;3657871859501171040;1782439090818545916;6936314038707413523;5117290135792467652;720840888466250585;8838717711606756251;9172976955054793469;6296485275518770064;8154150695030794787;8199832121759092112;773321667872991333;7270669313837600482;3408398779125901630;4359277201620147874;448184584262668803;9029095803741432716,1975590437749032870,3948283326616421003,3,3,4,14,4505772604969228686,1,1003,2005,3003,282924576738839389,1537236544,4006,5799347067982556520:-1;509660095530134768:-1;5755694407684602296:-1;8277336076276184272:9148482949976129397;7908382889764677758:-1,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0,2018-09-18","02:09:04,2018-09-18","10:09:04,18,10"
"1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;2636395404473730413;8070395809181082496;6434796455031995313;7839592306500064003;3657871859501171040;1782439090818545916;6936314038707413523;5117290135792467652;720840888466250585;8838717711606756251;9172976955054793469;6296485275518770064;8154150695030794787;8199832121759092112;773321667872991333;7270669313837600482;3408398779125901630;4359277201620147874;448184584262668803;9029095803741432716,1975590437749032870,3948283326616421003,3,3,4,14,2692638157208937547,0,1002,2005,3006,4007979028023783431,1537243232,4001,""5799347067982556520:9172976955054793469;7908382889764677758:1787573075717641245,9172976955054793469,5195139481388729954,4621934203383159480,9148482949976129397"",6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0,2018-09-18","04:00:32,2018-09-18","12:00:32,18,12"
"2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;2636395404473730413;8070395809181082496;6434796455031995313;7839592306500064003;3657871859501171040;1782439090818545916;6936314038707413523;5117290135792467652;720840888466250585;8838717711606756251;9172976955054793469;6296485275518770064;8154150695030794787;8199832121759092112;773321667872991333;7270669313837600482;3408398779125901630;4359277201620147874;448184584262668803;9029095803741432716,1975590437749032870,3948283326616421003,3,3,4,14,5247924392014515924,0,1003,2005,3004,4884875192608989870,1537211052,4001,5799347067982556520:5131280576272319091;7258015885215914736:5131280576272319091;7908382889764677758:5131280576272319091,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0,2018-09-17","19:04:12,2018-09-18","03:04:12,18,3"
"3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;2636395404473730413;8070395809181082496;6434796455031995313;7839592306500064003;3657871859501171040;1782439090818545916;6936314038707413523;5117290135792467652;720840888466250585;8838717711606756251;9172976955054793469;6296485275518770064;8154150695030794787;8199832121759092112;773321667872991333;7270669313837600482;3408398779125901630;4359277201620147874;448184584262668803;9029095803741432716,1975590437749032870,3948283326616421003,3,3,4,14,2681414445369714628,1,1004,2005,3006,840119421106178602,1537222670,4016,""509660095530134768:-1;5799347067982556520:-1;7908382889764677758:1787573075717641245,9148482949976129397;7492960463130085436:9148482949976129397;1950314698730389427:8218883658021718578,6641007758333551406,3749629542652371225,4038060334629950706,6165347051143749031